In [1]:
# TODO:
# * taking popularity into account

# Recommender system demo

This notebook implements a demo of *content-based* recommender system. 

Idea: create a vector for user based on his listening history (by summing 
normalized tracks vector). Calculate cosine similarities to determine tracks in 
database to recommend.

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [3]:
tracks_db = pd.read_csv('../data/features.csv', index_col='track_id')
tracks_db.head()

,title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artists_ids,genres
track_id,,,,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,NaN,0.653,0.796,7,-4.050,0,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,4,"[""6fOMl44jA4Sp5b9PpYCkzz""]","[[""hip hop"", ""pop rap"", ""rap""]]"
4ixBAI0E1M6df9mP27etZK,NaN,0.731,0.494,4,-9.567,1,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,4,"[""46VwdSiSccsgzNhJP9sGhQ"", ""2272CVpxVafY3L0UHu...","[[""icelandic hip hop"", ""icelandic pop""]]"
3jO5IAvslWxaTxswokg66i,NaN,0.859,0.464,3,-7.328,1,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,4,"[""2LRoIwlKmHjgvigdNGBHNo"", ""5J7rXWjtn5HzUkJ4Je...","[[""colombian pop"", ""latin"", ""pop reggaeton"", ""..."
5iS3TEmmRcDJU2pTJQ98zB,NaN,0.482,0.617,10,-10.288,0,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,4,"[""3MaQwUQBigrjcMYHk8bDKm"", ""5Y24oYaTX3VOvtAcG2...","[[""finnish dance pop"", ""finnish hip hop"", ""fin..."
29xPIFa0othUcFrEGyJCZu,NaN,0.824,0.684,1,-7.185,1,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,4,"[""2s7fZ34RK8RUJQ2ewsh6I5"", ""3tTvSeZiFDP3CY5EdP...","[[""francoton"", ""french hip hop"", ""pop urbaine""..."


### Version 1: using pure audio features only

In [4]:
feature_names = [
    'danceability', 'energy', 'key', 'loudness',	
    'mode',	'speechiness', 'acousticness', 'instrumentalness',
    'liveness',	'valence', 'tempo', 'duration_ms', 'time_signature']
tracks_db = tracks_db[feature_names]
tracks_db.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
track_id,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,0.653,0.796,7,-4.050,0,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,4
4ixBAI0E1M6df9mP27etZK,0.731,0.494,4,-9.567,1,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,4
3jO5IAvslWxaTxswokg66i,0.859,0.464,3,-7.328,1,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,4
5iS3TEmmRcDJU2pTJQ98zB,0.482,0.617,10,-10.288,0,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,4
29xPIFa0othUcFrEGyJCZu,0.824,0.684,1,-7.185,1,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,4


Categorical variables have to be represented properly.

In [5]:
cat_features = ['key', 'mode', 'time_signature']
for cat_feature in cat_features:
    dummies = pd.get_dummies(tracks_db[cat_feature], prefix=f'{cat_feature}_')
    tracks_db = pd.concat([tracks_db, dummies], axis=1)

tracks_db.drop(columns=cat_features, inplace=True)
tracks_db.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__9,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5
track_id,,,,,,,,,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,0.653,0.796,-4.050,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,...,0,0,0,1,0,0,0,0,1,0
4ixBAI0E1M6df9mP27etZK,0.731,0.494,-9.567,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,...,0,0,0,0,1,0,0,0,1,0
3jO5IAvslWxaTxswokg66i,0.859,0.464,-7.328,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,...,0,0,0,0,1,0,0,0,1,0
5iS3TEmmRcDJU2pTJQ98zB,0.482,0.617,-10.288,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,...,0,1,0,1,0,0,0,0,1,0
29xPIFa0othUcFrEGyJCZu,0.824,0.684,-7.185,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,...,0,0,0,0,1,0,0,0,1,0


All the features should be in the same range.

In [6]:
scaler = MinMaxScaler()
tracks_db_scaled = pd.DataFrame(
    scaler.fit_transform(tracks_db.to_numpy()), 
    columns=tracks_db.columns, index=tracks_db.index)
tracks_db_scaled.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__9,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5
track_id,,,,,,,,,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,0.662944,0.795214,0.836288,0.253623,0.241449,0.000000,0.096867,0.409505,0.695481,0.175277,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4ixBAI0E1M6df9mP27etZK,0.742132,0.492049,0.684643,0.048344,0.202213,0.000000,0.150113,0.482305,0.521722,0.155861,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3jO5IAvslWxaTxswokg66i,0.872081,0.461934,0.746186,0.299172,0.209256,0.000000,0.103010,0.374115,0.586569,0.130187,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
5iS3TEmmRcDJU2pTJQ98zB,0.489340,0.615524,0.664825,0.089959,0.779678,0.588889,0.095843,0.353893,0.756387,0.213194,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
29xPIFa0othUcFrEGyJCZu,0.836548,0.682782,0.750117,0.421325,0.023742,0.008616,0.064305,0.514661,0.613415,0.123509,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


Let's pretend we have a record of user's listening history (no timestamps).

In [7]:
tracks_user = tracks_db.sample(5)
tracks_user.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__9,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5
track_id,,,,,,,,,,,,,,,,,,,,,
1nqaSoiLuqevDes4S18sO7,0.845,0.787,-2.683,0.0374,0.2610,0.000003,0.1020,0.882,124.983,224997,...,0,0,0,0,1,0,0,0,1,0
4NIOFeptzAsbkOUnyBE6FY,0.715,0.600,-7.900,0.0344,0.3140,0.000006,0.0991,0.366,119.000,180748,...,0,0,0,0,1,0,0,0,1,0
7fTsw0IKbbPDN19keS32tF,0.853,0.777,-4.023,0.0684,0.1010,0.000615,0.3320,0.547,94.031,239373,...,0,0,0,1,0,0,0,0,1,0
3yQbBL8hsYtUhXSrmbKq8X,0.319,0.509,-8.868,0.0305,0.6420,0.000000,0.3900,0.427,144.550,223240,...,0,1,0,0,1,0,0,1,0,0
2h5vv3kvkeCxFjKUT7sLvu,0.661,0.699,-5.876,0.3020,0.0953,0.000000,0.1000,0.598,92.845,185983,...,0,0,0,0,1,0,0,0,1,0


In [17]:
def get_similarites(tracks_db_scaled, tracks_user, scaler):
    user_vector = tracks_user.to_numpy()
    user_vector_scaled = scaler.transform(user_vector)
    user_vector_scaled = np.sum(user_vector_scaled, axis=0, keepdims=True)

    return cosine_similarity(
        tracks_db_scaled, user_vector_scaled)

In [9]:
similarities = get_similarites(tracks_db_scaled, tracks_user, scaler)
track_db = tracks_db.assign(similarity=similarities)
track_db.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5,similarity
track_id,,,,,,,,,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,0.653,0.796,-4.050,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,...,0,0,1,0,0,0,0,1,0,0.728231
4ixBAI0E1M6df9mP27etZK,0.731,0.494,-9.567,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,...,0,0,0,1,0,0,0,1,0,0.841780
3jO5IAvslWxaTxswokg66i,0.859,0.464,-7.328,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,...,0,0,0,1,0,0,0,1,0,0.838337
5iS3TEmmRcDJU2pTJQ98zB,0.482,0.617,-10.288,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,...,1,0,1,0,0,0,0,1,0,0.708872
29xPIFa0othUcFrEGyJCZu,0.824,0.684,-7.185,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,...,0,0,0,1,0,0,0,1,0,0.888753


In [10]:
track_db.sort_values('similarity', ascending=False).iloc[:10]

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5,similarity
track_id,,,,,,,,,,,,,,,,,,,,,
2xjd6t95oEuxTn9vjWkC0n,0.918,0.814,-0.641,0.0582,0.449,0.000951,0.3780,0.743,128.003,204561,...,0,0,0,1,0,0,0,1,0,0.923281
5jiGCDUwv8gqWLpWMv4hok,0.904,0.905,-0.753,0.1570,0.233,0.000294,0.2410,0.796,120.089,196141,...,0,0,0,1,0,0,0,1,0,0.922718
6V2yiB8b0769W0MV1pQ95M,0.913,0.860,-1.383,0.1310,0.250,0.000226,0.2610,0.736,120.022,230000,...,0,0,0,1,0,0,0,1,0,0.922332
6WBEzul5kA06o2OR1pXqIt,0.766,0.905,-2.249,0.0676,0.353,0.000016,0.3400,0.820,135.019,217040,...,0,0,0,1,0,0,0,1,0,0.921959
1U3CZqgtekCchvUXV6o9l1,0.891,0.891,-2.139,0.0561,0.314,0.036000,0.2440,0.721,117.057,249733,...,0,0,0,1,0,0,0,1,0,0.921912
3pM81OKNrV6d2zJb76nQ6M,0.887,0.884,0.076,0.0716,0.404,0.007220,0.0895,0.963,130.009,206860,...,0,0,0,1,0,0,0,1,0,0.921635
3ds1VoDnWeSJ3XbkSamt94,0.871,0.931,-3.647,0.0503,0.305,0.002750,0.2320,0.767,125.030,225600,...,0,0,0,1,0,0,0,1,0,0.921422
38I8OemgIPYHdP5uBNY8BO,0.871,0.931,-3.647,0.0504,0.305,0.002750,0.2320,0.766,125.026,225600,...,0,0,0,1,0,0,0,1,0,0.921419
0j6LeC4rexb63tw2IaY5Gq,0.765,0.950,-2.409,0.0913,0.441,0.000013,0.3420,0.882,130.011,178619,...,0,0,0,1,0,0,0,1,0,0.921119


### Version 2: adding time

In [7]:
tracks_user = tracks_db.sample(5, random_state=7)
tracks_user.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__9,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5
track_id,,,,,,,,,,,,,,,,,,,,,
3ayMReTwLoo6jAg7adDqyz,0.828,0.790,-3.021,0.0719,0.16700,0.00001,0.3530,0.3790,104.016,247126,...,0,1,0,1,0,0,0,0,1,0
4YeP3VXJQbXzee7dSiGYS6,0.891,0.610,-5.177,0.3360,0.09680,0.00000,0.0931,0.5810,150.009,156028,...,0,1,0,1,0,0,0,0,1,0
0YQEWdfq3ajtNtK106Dr0k,0.632,0.691,-5.226,0.0603,0.00604,0.00149,0.1900,0.0768,116.028,159315,...,1,0,0,1,0,0,0,0,1,0
0Hq2gjlbDNlrd6J5Ixfgqs,0.658,0.705,-6.190,0.0253,0.01540,0.00399,0.1030,0.4130,100.021,215982,...,0,0,0,0,1,0,0,0,1,0
763v0n4szIsWp3HOsfi7qq,0.598,0.415,-7.081,0.0668,0.19900,0.00000,0.1110,0.2990,120.151,205295,...,0,0,0,0,1,0,0,0,1,0


In [10]:
def get_similarites(tracks_db_scaled, tracks_user, scaler):
    user_vector = tracks_user.to_numpy()
    user_vector = scaler.transform(user_vector)

    beta = 0.9

    # 1st method - without bias correction
    weights = np.logspace(0, user_vector.shape[0]-1, 
                          num=user_vector.shape[0], base=beta)
    weights *= 1-beta
    weights = weights[::-1].reshape(-1, 1)
    user_vector *= weights
    user_vector = np.sum(user_vector, axis=0, keepdims=True)

    return cosine_similarity(
        tracks_db_scaled, user_vector)

In [8]:
def get_similarites(tracks_db_scaled, tracks_user, scaler):
    user_vector = tracks_user.to_numpy()
    user_vector = scaler.transform(user_vector)

    beta = 0.9

    # 1st method - without bias correction   
    user_vector_weighted = 0
    for i in range(0, user_vector.shape[0]):
        user_vector_weighted = (
            (beta*user_vector_weighted + (1-beta)*user_vector[i]))   

    user_vector_weighted = user_vector_weighted.reshape(1, -1)

    return cosine_similarity(
        tracks_db_scaled, user_vector_weighted)

In [11]:
similarities = get_similarites(tracks_db_scaled, tracks_user, scaler)
track_db = tracks_db.assign(similarity=similarities)
track_db.sort_values('similarity', ascending=False).iloc[:10]

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5,similarity
track_id,,,,,,,,,,,,,,,,,,,,,
5GF774YRVWL7rDVjYCPFla,0.900,0.841,-0.872,0.188,0.0944,0.049600,0.1390,0.548,121.041,183876,...,1,0,1,0,0,0,0,1,0,0.906262
7214sQpmw7nAgTfE4D3vQt,0.888,0.834,-1.392,0.177,0.0819,0.019500,0.1980,0.557,121.015,184827,...,1,0,1,0,0,0,0,1,0,0.906115
6csZYoffpZ7iuSw83x2zVy,0.913,0.788,-2.889,0.263,0.0546,0.000640,0.1680,0.544,120.934,143314,...,1,0,1,0,0,0,0,1,0,0.902922
1SeWEXrrqObbQH9vKMrWsb,0.896,0.795,-3.164,0.113,0.0376,0.006180,0.0644,0.573,131.010,199236,...,1,0,1,0,0,0,0,1,0,0.902826
7KOlJ92bu51cltsD9KU5I7,0.865,0.687,-3.744,0.217,0.2420,0.000000,0.1580,0.486,139.023,242407,...,1,0,1,0,0,0,0,1,0,0.902306
1vBeIXlzbAieGoDqInav5j,0.947,0.788,-6.138,0.124,0.1070,0.000691,0.1080,0.454,129.977,229005,...,1,0,1,0,0,0,0,1,0,0.902087
4GeqhagYz03kwTAbKBaVf9,0.868,0.855,-5.655,0.144,0.0935,0.000182,0.1060,0.518,130.014,170000,...,1,0,1,0,0,0,0,1,0,0.901782
60SdxE8apGAxMiRrpbmLY0,0.876,0.786,-4.884,0.306,0.0627,0.000000,0.1270,0.575,153.068,244679,...,1,0,1,0,0,0,0,1,0,0.901563
4LIZDbjLNK9N0fugJ0z4Pf,0.961,0.783,-5.191,0.194,0.1950,0.000000,0.0979,0.589,128.019,316253,...,1,0,1,0,0,0,0,1,0,0.901531


In [8]:
def get_similarites(tracks_db_scaled, tracks_user, scaler):
    user_vector = tracks_user.to_numpy()
    user_vector = scaler.transform(user_vector)

    beta = 0.9

    # 2nd method - with bias correction   
    user_vector_weighted = 0
    for t in range(1, user_vector.shape[0]+1):
        user_vector_weighted = (
            (beta*user_vector_weighted + (1-beta)*user_vector[t-1]) / (1-beta**t))    

    user_vector_weighted = user_vector_weighted.reshape(1, -1)

    return cosine_similarity(
        tracks_db_scaled, user_vector_weighted)

In [16]:
similarities = get_similarites(tracks_db_scaled, tracks_user, scaler)
tracks_db = tracks_db.assign(similarity=similarities)

# Drop tracks that are already there in user history
# tracks_db = tracks_db.drop(tracks_user.index)

tracks_db.sort_values('similarity', ascending=False).iloc[:10]

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5,similarity
track_id,,,,,,,,,,,,,,,,,,,,,
3ayMReTwLoo6jAg7adDqyz,0.828,0.790,-3.021,0.0719,0.1670,0.000010,0.353,0.379,104.016,247126,...,1,0,1,0,0,0,0,1,0,0.999653
3HF2E09nlXRlLQO2TJtQJh,0.752,0.743,-5.006,0.0749,0.1420,0.000000,0.256,0.386,116.013,177040,...,1,0,1,0,0,0,0,1,0,0.998425
3znlxKwQTRVrrerCcZPus2,0.828,0.714,-6.852,0.1080,0.1850,0.000003,0.244,0.407,95.967,143179,...,1,0,1,0,0,0,0,1,0,0.997479
6WkJ2OK163XXS2oARUC9JM,0.720,0.741,-6.529,0.0287,0.1010,0.000014,0.289,0.357,119.896,198053,...,1,0,1,0,0,0,0,1,0,0.997071
3AqwdLkl62kRTE8TwXxeIM,0.730,0.748,-5.609,0.1650,0.1300,0.000000,0.348,0.305,109.969,145187,...,1,0,1,0,0,0,0,1,0,0.997028
5KPKjcE2trO2hBWRzkb2Mo,0.781,0.854,-4.115,0.0740,0.0263,0.000216,0.344,0.410,103.021,160552,...,1,0,1,0,0,0,0,1,0,0.996928
3S9U5VgWx3CT0xUuhaufEK,0.782,0.902,-4.335,0.0615,0.2170,0.000000,0.374,0.434,129.944,194341,...,1,0,1,0,0,0,0,1,0,0.996802
0A5gdlrpAuQqZ2iFgnqBFW,0.813,0.761,-5.687,0.0850,0.0529,0.011900,0.337,0.290,124.957,173067,...,1,0,1,0,0,0,0,1,0,0.996785
3IZRWNhxHu5mrbOOiboZcr,0.777,0.691,-6.719,0.0548,0.2220,0.000124,0.249,0.460,122.069,233115,...,1,0,1,0,0,0,0,1,0,0.996529


### Version 3: adding popularity

In [ ]:
tracks_user = tracks_db.sample(5)
tracks_user.head()

In [ ]:
def get_similarites(tracks_db_scaled, tracks_user, scaler, 
        popularity_rate=1):
    user_vector = tracks_user.to_numpy()
    user_vector = scaler.transform(user_vector)

    beta = 0.9

    # 2nd method - with bias correction   
    user_vector_weighted = 0
    for i in user_vector.shape[0]:
        user_vector_weighted = (
            (beta*user_vector_weighted + (1-beta)*user_vector[i]) / (1-beta*i))    

    similarity = cosine_similarity(
        tracks_db_scaled, user_vector_weighted)
    popularitiyes = track_db_scaled.
    similiraty += 